# Precinct Matching Framework

In [ ]:
import pandas as pd
import geopandas as gpd

### Import the datasets

In [ ]:
county_id = 'Ulster County'

county_results_filename = 'election_results_county_id={}.csv'.format(county_id)
county_results_df = pd.read_csv(county_results_filename)
county_shapefile_filename = 'shapefile_county_id={}'.format(county_id)
county_shapefile_gdf = gpd.read_file(county_shapefile_filename)

# correct for the truncation caused by 10 character column name limit in shapefiles
county_shapefile_gdf.rename(columns={'original_p':'original_precinct_name'}, inplace=True)

The next cell aliases `county_results_df` as `df` and `county_shapefile_gdf` as `gdf` here because typing fewer characters allows for faster data exploration. 

In [ ]:
df = county_results_df.copy()
gdf = county_shapefile_gdf.copy()

In [ ]:
df.head()

In [ ]:
gdf.head()

### Check Preconditions
These should all pass - they're here to ensure that everything in `config.ipynb` worked correctly

In [ ]:
# TODO: Pass the precondition described above which takes the form of an assert statement in this cell.
assert 'county_id' in df.columns and 'county_id' in gdf.columns
assert 'original_precinct_name' in df.columns and 'original_precinct_name' in gdf.columns

### General Modifications
Its normally benificial to apply some modifications uniformly to all precincts. For example, its good practice to make everything lower case. This modification is made in `edit_precinct_name` - read its specification to learn more about how to use it to make more modifications. Consider removing substrings that appear in every precinct name like `voting district`. Be careful of removing words that will result in duplicate precinct names. For example, if there are two `Newtown Voting District` precincts in the shapefile, and the election results have `Newtown Boro` and `Newtown Township`, its okay to remove `Voting District`, but you probably don't want to remove `Boro` or `Township`. Of course, this will differ from County to County - so be vigilant!

In [19]:
def edit_precinct_name(prec_name, 
    remove_lst=[], 
    target_to_replacement={},
    stopping_words=[],
    prec_dict={}):
    '''
    Returns a lower case precinct name (string) with certian modifications depending other arguments. 
    
    Modifications are performed in order of the parameters they depend on. By convention, case is 
    ignored by making prec_name lower case. Accordingly, one should pass arguements with lower case
    elements. That is, keys of the dictionaries and elements of lists should be lower case strings.

	Parameters:
		prec_name (str): precinct name
		remove_lst ((str) list): if a string in this list is a substring in prec_name it will be removed. 
            All elements should be lower case.
        target_to_replacement ({str:str} dictionary): keys (targets) will be replaced with their 
            corresponding value (replacements) in prec_name. All keys should be lower case.
        stopping_words ({str} list): If any substring of prec_name contains a element of stopping_words
             that is adjacent to a space character it will be removed. All elements should be lower case.
        prec_dict ({str:str} dictionary): After all the modifications above, if the edited prec_name
            string is in the set of keys for prec_dict, then it will be replaced with that key's value.  
            All keys should be lower case.

	Returns:
		prec_name (str): prec_name arguement returned with the 
    '''
    prec_name = str(prec_name).lower()
    for word in remove_lst:
        prec_name = prec_name.replace(word, '')
    for target, replacement in target_to_replacement.items():
        prec_name = prec_name.replace(target, replacement)
    words = prec_name.split()
    words = [word.lstrip('0') for word in words if word not in stopping_words]
    prec_name = " ".join(words)
    return prec_dict[prec_name] if prec_name in prec_dict.keys() else prec_name

df_to_gdf = {
}

gdf_to_df = {
}

# Tune the matching by adding optional arguements to edit
df['edited_precinct_name'] = df['original_precinct_name'].apply(lambda name: edit_precinct_name(name,prec_dict=df_to_gdf))
gdf['edited_precinct_name'] = gdf['original_precinct_name'].apply(lambda name: edit_precinct_name(name,prec_dict=gdf_to_df))

######## Manual Corrections ###########
# Make precinct specific corrections here like splitting one precinct into two because of new congressional districts
gdf.loc[gdf['VTDST']=='000290','edited_precinct_name'] = 'howard township'
gdf.loc[gdf['VTDST']=='000300','edited_precinct_name'] = 'howard borough'

# make the 'original_precinct_name, edited_precinct_name' for use in the loop below
df['original_precinct_name, edited_precinct_name'] = df[['original_precinct_name','edited_precinct_name']].apply(tuple, axis=1)
gdf['original_precinct_name, edited_precinct_name'] = gdf[['original_precinct_name','edited_precinct_name']].apply(tuple, axis=1)

######## Matching Framework ###########
unmatched_precinct_lst_gdf = []
unmatched_precinct_lst_df = []

precinct_list_df = sorted(list(df[df['county_id'] == county_id]['original_precinct_name, edited_precinct_name'].unique()), key=lambda x: x[1])
precinct_list_gdf = sorted(list(gdf[gdf['county_id'] == county_id]['original_precinct_name, edited_precinct_name'].unique()), key=lambda x: x[1])

precinct_set_df = {x[1] for x in precinct_list_df if x[1] not in unmatched_precinct_lst_df}
precinct_set_gdf = {x[1] for x in precinct_list_gdf if x[1] not in unmatched_precinct_lst_gdf}

unmatched_precincts_df = sorted(list(precinct_set_df - precinct_set_gdf))
unmatched_precincts_gdf = sorted(list(precinct_set_gdf - precinct_set_df))
n_unmatched = len(unmatched_precincts_df) + len(unmatched_precincts_gdf)
if n_unmatched > 0:
        print("county_id: '{}' | {} precincts in {} | {} precincts in {}:\n".format(county_id, len(precinct_list_df), dataset_name_df, len(precinct_list_gdf), dataset_name_gdf))
        n_precincts_total = len(precinct_list_df) + len(precinct_list_gdf)
        print(n_unmatched, " precincts are unmatched out of ", n_precincts_total)
        df_unmatched = df[(df['edited_precinct_name'].isin(unmatched_precincts_df)) & (df.county_id == county_id)]
        gdf_unmatched = gdf[(gdf['edited_precinct_name'].isin(unmatched_precincts_gdf)) & (gdf.county_id == county_id)]
        if n_unmatched > 100:
            print("\nLook for parterns and use change the parameters to edit_precinct_name accordingly.\n")
            for index, (original_precinct_name_df, edited_precinct_name_df) in enumerate(precinct_list_df):
                original_precinct_name_gdf, edited_precinct_name_gdf = precinct_list_gdf[index]
                if edited_precinct_name_df in unmatched_precincts_df and edited_precinct_name_gdf in unmatched_precincts_gdf:
                    print("{} <-- {} ({})".format(edited_precinct_name_df, original_precinct_name_df, dataset_name_df))
                    print("{} <-- {} ({})\n".format(edited_precinct_name_gdf, original_precinct_name_gdf, dataset_name_gdf))
        else:
            print("unmatched_precincts_df ({}) - len = {}| '{}':".format(dataset_name_df, len(unmatched_precincts_df), county_id), unmatched_precincts_df)
            print("\nunmatched_precincts_gdf ({}) - len = {}| '{}':".format(dataset_name_gdf, len(unmatched_precincts_gdf), county_id), unmatched_precincts_gdf)
            precinct_modification_dictionary_df_to_gdf = {unmatched_precincts_df[i]: unmatched_precincts_gdf[i] if i < len(unmatched_precincts_gdf) else '' for i in range(len(unmatched_precincts_df))}
            precinct_modification_dicitonary_gdf_to_df = {unmatched_precincts_gdf[i]: unmatched_precincts_df[i] if i < len(unmatched_precincts_df) else '' for i in range(len(unmatched_precincts_gdf))}
            print("{}  to {} precinct modification dictionary: ".format(dataset_name_df, dataset_name_gdf))
            print("'{}':".format(county_id))
            pprint(precinct_modification_dictionary_df_to_gdf)
            print("{}  to {} precinct modification dictionary: ".format(dataset_name_gdf, dataset_name_df))
            print("'{}':".format(county_id))
            pprint(precinct_modification_dicitonary_gdf_to_df)
            for index, (original_precinct_name_df, edited_precinct_name_df) in enumerate(precinct_list_df):
                original_precinct_name_gdf, edited_precinct_name_gdf = precinct_list_gdf[index]
                if edited_precinct_name_df in unmatched_precincts_df or edited_precinct_name_gdf in unmatched_precincts_gdf:
                    print("{} <-- {} ({})".format(edited_precinct_name_df, original_precinct_name_df, dataset_name_df))
                    print("{} <-- {} ({})\n".format(edited_precinct_name_gdf, original_precinct_name_gdf, dataset_name_gdf))
            else:
                print("Add unmatched precincts to the unmatched precinct.")
else:
    print("All Done! (make sure you have one to one matches)")